In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss
from sklearn.metrics import zero_one_loss

from xgboost import XGBClassifier

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

In [3]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [4]:
ws.datasets

{'DummyPredictionTrain': DatasetRegistration(id='723a8324-8b50-48ba-9398-53c659959f56', name='DummyPredictionTrain', version=5, description='', tags={}), 'DummyPredictionTest': DatasetRegistration(id='b217579b-16ba-4c90-9e95-1776ba592742', name='DummyPredictionTest', version=6, description='', tags={}), 'DummyPredictionPipeline': DatasetRegistration(id='75fbd0e5-81c3-4386-a9e5-0658d3d16360', name='DummyPredictionPipeline', version=3, description='', tags={}), 'ProductPredictionTrain': DatasetRegistration(id='576ee7cb-1c73-45b7-8795-dab72c0d19d1', name='ProductPredictionTrain', version=1, description='', tags={}), 'ProductPredictionTest': DatasetRegistration(id='0ea979fb-10aa-4c82-a7cc-1623ca178f22', name='ProductPredictionTest', version=1, description='', tags={}), 'symptomcodes.csv': DatasetRegistration(id='88af5740-1a1b-4e09-8129-d3c538680909', name='symptomcodes.csv', version=2, description='', tags={}), 'ItemResourceData.csv': DatasetRegistration(id='02e6cb83-4d0c-42b2-bbef-e103c74

In [5]:
# df = pd.read_csv('WILO_traindata.csv', sep=';', header=0)
df = ws.datasets['ProductPredictionTrain'].to_pandas_dataframe()
print(len(df))
df.head()
# df['CaseId'] = df['Job Card.JobCard Number']


KeyboardInterrupt



In [24]:
df = df.iloc[10000:20000]

In [ ]:
# for c in df.columns:
#     print(c)

### Feature / Target Split

In [ ]:
X = df.drop([ c for c in df.columns if c.startswith('product_')], axis=1).drop(['CaseId'], axis=1).values
y = df[[ c for c in df.columns if c.startswith('product_')]].values

In [ ]:
y_clean = []
# remove columns with too few entries for target
for i in range(y.shape[1]):
    if y[:,i].sum() >= 5:
        y_clean.append(y[:,i])
y_clean = np.transpose(np.array(y_clean))

In [28]:
y_clean.shape

(0,)

In [29]:
# n = []
# for comp in range(1,1100, 10):
#     pca = PCA(n_components=comp)
#     X_tr = pca.fit_transform(X)
#     n.append([comp, pca.explained_variance_ratio_.sum()])

In [30]:
# import matplotlib.pyplot as plt
# plt.scatter(np.array(n)[:,0], np.array(n)[:,1])

In [31]:
pca = PCA(n_components=150)
X_tr = pca.fit_transform(X)
pca.explained_variance_ratio_.sum()

ValueError: could not convert string to float: 'C-BE008616-CP JC02.00'

### Train Test Split

##### Idee/Bedenken: evtl Zeitreihe --> nicht train-test-split sd. "neueste" Datensätze

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tr, y_clean, test_size=0.1, random_state=42)

In [ ]:
y_test[:,64].sum()

In [ ]:
for i in range(y_train.shape[1]):
    if y_train[:,i].sum() == 0:
        print('###########')
        print(i)
        print(y_train[:,i].sum())
        print(y_test[:,i].sum())
        print('###########')

In [ ]:
for i in range(y_test.shape[1]):
    if y_test[:,i].sum() == 0:
        print('###########')
        print(i)
        print(y_test[:,i].sum())
        print(y_train[:,i].sum())
        print('###########')

## Train

### SGD Classifier

In [ ]:
clf = MultiOutputClassifier(SGDClassifier(max_iter=1000, tol=1e-3)) # n_jobs
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

### Evaluation

In [ ]:
zero_one_loss(y_test, y_pred)

In [ ]:
hamming_loss(y_test,y_pred)

### XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
clf = MultiOutputClassifier(XGBClassifier(n_jobs=-1, max_depth=4))
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
zero_one_loss(y_test, y_pred)

In [ ]:
hamming_loss(y_test,y_pred)

### SGD Classifier

In [ ]:
clf = OneVsRestClassifier(SGDClassifier(max_iter=1000, tol=1e-3)) # n_jobs
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

### Evaluation

In [ ]:
zero_one_loss(y_test, y_pred)

In [ ]:
hamming_loss(y_test,y_pred)

### XGBoost

In [ ]:
clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
zero_one_loss(y_test, y_pred)

In [ ]:
hamming_loss(y_test,y_pred)

### Keras

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_clean, test_size=0.1, random_state=42)

In [26]:
X_train = np.array([[1,0,1,1,0],[1,1,1,0,0]])
X_test = np.array([[1,0,1,1,0],[1,1,1,0,0]])
y_train = np.array([[1.,0.,1.,1.,0.],[1.,1.,1.,0.,0.]])
y_test = np.array([[1.,0.,1.,1.,0.],[1.,1.,1.,0.,0.]])

In [8]:
y_test.shape

(2, 5)

In [9]:
# pip install tensorflow-gpu

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense

In [11]:
inputs = Input(shape=(X_train.shape[1],))

# x = Dense(1024, activation='relu')(inputs)
x = Dense(512, activation='relu')(inputs)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

outputs = Dense(y_train.shape[1], activation='sigmoid')(x)

In [12]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 512)               3072      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 165   

In [13]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=["accuracy"],
)

history = model.fit(X_train, y_train, batch_size=64, epochs=100)

Train on 2 samples
Epoch 1/100
2/2 [==============================] - 3s 1s/sample - loss: 0.6940 - accuracy: 0.4000
Epoch 2/100
2/2 [==============================] - 0s 2ms/sample - loss: 0.6423 - accuracy: 0.8000
Epoch 3/100
2/2 [==============================] - 0s 2ms/sample - loss: 0.5760 - accuracy: 0.8000
Epoch 4/100
2/2 [==============================] - 0s 2ms/sample - loss: 0.5027 - accuracy: 0.8000
Epoch 5/100
2/2 [==============================] - 0s 2ms/sample - loss: 0.4357 - accuracy: 0.8000
Epoch 6/100
2/2 [==============================] - 0s 2ms/sample - loss: 0.3833 - accuracy: 0.8000
Epoch 7/100
2/2 [==============================] - 0s 1ms/sample - loss: 0.3314 - accuracy: 0.9000
Epoch 8/100
2/2 [==============================] - 0s 1ms/sample - loss: 0.2888 - accuracy: 1.0000
Epoch 9/100
2/2 [==============================] - 0s 2ms/sample - loss: 0.2517 - accuracy: 1.0000
Epoch 10/100
2/2 [==============================] - 0s 1ms/sample - loss: 0.2150 - accuracy

Epoch 82/100
2/2 [==============================] - 0s 2ms/sample - loss: 5.0304e-05 - accuracy: 1.0000
Epoch 83/100
2/2 [==============================] - 0s 2ms/sample - loss: 4.6881e-05 - accuracy: 1.0000
Epoch 84/100
2/2 [==============================] - 0s 1ms/sample - loss: 4.3591e-05 - accuracy: 1.0000
Epoch 85/100
2/2 [==============================] - 0s 1ms/sample - loss: 4.0673e-05 - accuracy: 1.0000
Epoch 86/100
2/2 [==============================] - 0s 1ms/sample - loss: 3.7902e-05 - accuracy: 1.0000
Epoch 87/100
2/2 [==============================] - 0s 2ms/sample - loss: 3.5297e-05 - accuracy: 1.0000
Epoch 88/100
2/2 [==============================] - 0s 2ms/sample - loss: 3.2933e-05 - accuracy: 1.0000
Epoch 89/100
2/2 [==============================] - 0s 2ms/sample - loss: 3.0650e-05 - accuracy: 1.0000
Epoch 90/100
2/2 [==============================] - 0s 2ms/sample - loss: 2.8610e-05 - accuracy: 1.0000
Epoch 91/100
2/2 [==============================] - 0s 2ms/sampl

In [27]:
y_pred = model.predict(X_test)

In [31]:
np.round(y_pred)

array([[1., 0., 1., 1., 0.],
       [1., 1., 1., 0., 0.]], dtype=float32)

In [34]:
y_pred.round()

array([[1., 0., 1., 1., 0.],
       [1., 1., 1., 0., 0.]], dtype=float32)

In [35]:
zero_one_loss(y_test, y_pred.round())

0.0

In [33]:
hamming_loss(y_test,y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets